# Ray RLlib Multi-Armed Bandits - Market Bandit Example

© 2019-2021, Anyscale. All Rights Reserved

![Anyscale Academy](../../images/AnyscaleAcademyLogo.png)

Now that we've learned about multi-armed bandits and methods for optimizing rewards, let's look at real-world applications, starting with a stock market example.

How well could you invest in the public markets, if you could only observe one macroeconomic signal *inflation* and could only update your investments once each year?

To explore this, first we'll load a dataset derived from this [NYU Stern table](http://pages.stern.nyu.edu/~adamodar/New_Home_Page/datafile/histretSP.html) that shows returns for nearly a century of market data, including dividends and adjustments for inflation. The `market.tsv` file in this folder contains the data.

In [ ]:
import pandas as pd
import numpy as np
import os
import sys

In [ ]:
# Some properties we'll need:
DEFAULT_MAX_INFLATION = 100.0
DEFAULT_TICKERS = ["sp500", "t.bill", "t.bond", "corp"]
DEFAULT_DATA_FILE = os.path.abspath(os.path.curdir) + "/market.tsv"  # full path

In [ ]:
def load_market_data (file_name):
    with open(file_name, "r") as f:
        return pd.read_table(f)

Let's load and examine the data:

In [ ]:
df = load_market_data(DEFAULT_DATA_FILE)
df

As you can see the data spans 92 years, from 1928 to 2019. 

The columns represent:
  * `year`: the year
  * `inflation`: the inflation rate at the time
  * `sp500`: [S&P500](https://en.wikipedia.org/wiki/S%26P_500_Index) (composite stock index)
  * `t.bill`: [Treasury Bills](https://www.investopedia.com/terms/t/treasurybill.asp) (short-term gov bonds)
  * `t.bond`: [Treasury Bonds](https://www.investopedia.com/terms/t/treasurybond.asp) (long-term gov bonds)
  * `corp`: [Moody's Baa Corporate Bonds](https://en.wikipedia.org/wiki/Moody%27s_Investors_Service#Moody's_credit_ratings) (moderate risk)

## Analysis of the Data

Let's also look at descriptions statistics for each column:

In [ ]:
df.describe()

What are the worst case and best case scenarios? In other words, if one could predict the future market performance, what are the possible ranges of total failure vs. total success over the past century? By "total", we mean what if you had all your money in a given year invested in the worst performing _sector_ (S&P500, T bills, or other) or you were invested in the best performing sector for that year.

In [ ]:
n_years = len(df)
min_list = []
max_list = []

for i in range(n_years):
    row = df.iloc[i, 2:]
    min_list.append(min(row))
    max_list.append(max(row))
    
print("{:5.2f}% worst case annualized".format(sum(min_list) / n_years))
print("{:5.2f}% best case annualized".format(sum(max_list) / n_years))

In [ ]:
min_max = pd.DataFrame.from_dict({'year': df['year'], 'min':min_list, 'max':max_list})
min_max

We can visualize the best and worst returns, year over year.
Overall this should look like a [*random walk*](https://en.wikipedia.org/wiki/Random_walk):

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt

plt.fill_between(
    df["year"],
    min_list,
    max_list,
    color="b",
    alpha=0.2
)

plt.title("Best vs. Worst Market Return")
plt.xlabel("Year")
plt.ylabel("Return")
plt.show()

There are some years where the performance varies widely, while other years everything returns about the same performance.

## Defining an Environment

Now let's define a Gym environment so that we can train a contextual bandit to optimize annual investments over that period.

In [ ]:
import gym
from gym.spaces import Discrete, Box
from gym.utils import seeding
import random

This is the bandit we'll use to represent the market "environment".

In [ ]:
class MarketBandit (gym.Env):
    
    def __init__ (self, config={}):
        self.max_inflation = config.get("max-inflation", DEFAULT_MAX_INFLATION)
        self.tickers = config.get("tickers", DEFAULT_TICKERS)
        self.data_file = config.get("data-file", DEFAULT_DATA_FILE)
        print(f"MarketBandit: max_inflation: {self.max_inflation}, tickers: {self.tickers}, data file: {self.data_file} (config: {config})")

        self.action_space = Discrete(4)
        self.observation_space = Box(
            low  = -self.max_inflation,
            high =  self.max_inflation,
            shape=(1, )
        )
        self.df = load_market_data(self.data_file)
        self.cur_context = None


    def reset (self):
        self.year = self.df["year"].min()
        self.cur_context = self.df.loc[self.df["year"] == self.year]["inflation"][0]
        self.done = False
        self.info = {}

        return [self.cur_context]


    def step (self, action):
        if self.done:
            reward = 0.
            regret = 0.
        else:
            row = self.df.loc[self.df["year"] == self.year]

            # calculate reward
            ticker = self.tickers[action]
            reward = float(row[ticker])

            # calculate regret
            max_reward = max(map(lambda t: float(row[t]), self.tickers))
            regret = round(max_reward - reward)

            # update the context
            self.cur_context = float(row["inflation"])

            # increment the year
            self.year += 1

            if self.year >= self.df["year"].max():
                self.done = True

        context = [self.cur_context]
        #context = self.observation_space.sample()

        self.info = {
            "regret": regret,
            "year": self.year
        }
         
        return [context, reward, self.done, self.info]


    def seed (self, seed=None):
        """Sets the seed for this env's random number generator(s).
        Note:
            Some environments use multiple pseudorandom number generators.
            We want to capture all such seeds used in order to ensure that
            there aren't accidental correlations between multiple generators.
        Returns:
            list<bigint>: Returns the list of seeds used in this env's random
              number generators. The first value in the list should be the
              "main" seed, or the value which a reproducer should pass to
              'seed'. Often, the main seed equals the provided 'seed', but
              this won't be true if seed=None, for example.
        """
        self.np_random, seed = seeding.np_random(seed)
        return [seed]

Let's see it in action:

In [ ]:
bandit = MarketBandit()
bandit.reset()

for i in range(10):
    action = bandit.action_space.sample()
    obs = bandit.step(action)
    print(action, obs)

We can use this environment in a kind of [*monte carlo simulation*](https://en.wikipedia.org/wiki/Monte_Carlo_method) to measure a baseline for what the rewards would be over a long period if you merely used actions selected at random.

In [ ]:
done = 1
reward_list = []
iterations = 10000 #50000

for i in range(iterations):
    if done == 1:
        bandit.reset()

    action = bandit.action_space.sample()
    obs = bandit.step(action)
    context, reward, done, info = obs
    reward_list.append(reward)
    #print(action, context, reward, done, info)

In [ ]:
df_mc = pd.DataFrame(reward_list, columns=["reward"])
df_mc.mean()

Depending on the number of iterations, you'll probably get a value approaching 3.75% as a baseline for random actions. That's more than the -5.64% worst case and must less than 15.18% best case for the reward!

In [ ]:
df_mc.plot(y="reward", title="Reward Over Iterations")

([image](../../images/rllib/MarketReward-Random.png))

Yes, it looks quite random... There is no improvement (i.e., *learning*) happening at all.

## Training a policy in RLlib

Now let's train a policy using our contextual bandit, specifically using _Linear Thompson Sampling_ in RLlib. Hopefully it will do better than the random results we just computed!

Recall in the `__init__()` method for `MarketBandit` that we set some parameters from the passed in `config` object. 
So we need to create a custom config object with our parameters, by building on the default `TS_CONFIG` object for _LinTS_:

In [ ]:
import copy
from ray.rllib.contrib.bandits.agents.lin_ts import TS_CONFIG

market_config = copy.deepcopy(TS_CONFIG)

market_config["env"] = MarketBandit
market_config["max-inflation"] = DEFAULT_MAX_INFLATION;
market_config["tickers"] = DEFAULT_TICKERS;
market_config["data-file"] = DEFAULT_DATA_FILE;

We'll also define a custom trainer, which builds on the `LinTSTrainer` with "updates". 
This will be the first argument that we'll pass to `ray.tune.run()` later. 

Note: if all we needed was the default `LinTSTrainer` trainer, as is and with no customized config settings, we could instead just pass the string `"contrib/LinTS"` to `ray.tune.run()`.  

In [ ]:
from ray.rllib.contrib.bandits.agents.lin_ts import LinTSTrainer

MarketLinTSTrainer = LinTSTrainer.with_updates(
    name="MarketLinTSTrainer",
    default_config=market_config,      # Will be merged with Trainer.COMMON_CONFIG (rllib/agent/trainer.py)
    #default_policy=[somePolicyClass]  # If we had a policy...
)

Then initialize Ray:

In [ ]:
import ray

ray.init(ignore_reinit_error=True)

Then run Tune:

In [ ]:
stop = {
    "training_iteration": 100
}

analysis = ray.tune.run(
    MarketLinTSTrainer,
    config=market_config,
    stop=stop,
    num_samples=3,    
    checkpoint_at_end=True,
    verbose=1
)

In [ ]:
stats = analysis.stats()
secs = stats["timestamp"] - stats["start_time"]
print(f'{secs:7.2f} seconds, {secs/60.0:7.2f} minutes')

## Analyzing the results

Let's analyze the rewards and cumulative regrets from these trials.

In [ ]:
df_ts = pd.DataFrame()

for key, df_trial in analysis.trial_dataframes.items():
    df_ts = df_ts.append(df_trial, ignore_index=True)
    
df_ts.head()

In [ ]:
rewards = df_ts \
    .groupby("info/num_steps_trained")["episode_reward_mean"] \
    .aggregate(["mean", "max", "min", "std"])

rewards

In [ ]:
rewards.plot(y=["mean", "max"], secondary_y=True, title="Rewards vs. Steps")

The rewards bounce around at first, then appear to stabilize after 5000 steps, with slow improvement afterwards.

In [ ]:
regrets = df_ts \
    .groupby("info/num_steps_trained")["info/learner/default_policy/cumulative_regret"] \
    .aggregate(["mean", "max", "min", "std"])

regrets

In [ ]:
regrets.plot(y="mean", yerr="std", title="Regrets vs. Steps")

## Evaluating the Trained Policy

Overall, how well did the trained policy perform? The results should be better than random, but less than the best case.

In [ ]:
print("{:5.2f}% optimized return annualized".format(max(rewards["mean"]) / n_years))

You should see a number between near 6%. That's better than the random action baseline of 3.75%, but no where near the best case scenario of 15.18% return. Hence, our regrets continue to grow over time...

Note that investing solely in the S&P stock index which would have produced better than 8% return over that period -- that is, if one could wait 92 years. However, investing one's entire portfolio into stocks can become quite a risky policy in the short-term, so we were exploring how to balance a portfolio given only limited information.

In any case, the contextual bandit performed well considering that it could only use *inflation* for the context of its decisions, and could only take actions once each year.

## Exercise 1

Try using a `LinUCBTrainer`-based trainer.

How does the annualized return compare?

## Exercise 2

Inflation rates tend to get reported months after they've occurred. To be more accurate with using this dataset, offset the *inflation* observation one step (1 year) ahead.

How does the annualized return compare?

---

## Extra - Restoring from a Checkpoint

In the previous lesson, [05 Thompson Sampling](05-Thompson-Sampling.ipynb), we showed how to restore a trainer from a checkpoint, but almost "in passing". Let's use this feature again, this time with our custom trainer class `MarketLinTSTrainer`.

In [ ]:
trial = analysis.trials[0]
path = trial.checkpoint.value
print(f'checkpoint_path: {path}')

In [ ]:
trainer = MarketLinTSTrainer(market_config)  # create instance and then restore from checkpoint
trainer.restore(path)

Access the model, to review the distribution of arm weights

In [ ]:
model = trainer.get_policy().model
means = [model.arms[i].theta.numpy() for i in range(3)]
covs = [model.arms[i].covariance.numpy() for i in range(3)]
means, covs, model.arms[0].theta.numpy()

A final note: using checkpoints will change how the training performs in this notebook, if you rerun it. So be sure to start from scratch when doing experiments here, if that's what you intend!

In [ ]:
ray.shutdown()